In [1]:
import time
import cv2
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from numpy import random
import numpy as np
import math
import torchvision
import re

In [2]:
!pip install easyocr --quiet
import easyocr

In [3]:
%cd yolov7

/Users/jayanth.kumar/Desktop/work/Hackverse/GTSRB/yolov7


In [5]:
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7

fatal: destination path 'yolov7' already exists and is not an empty directory.
/Users/jayanth.kumar/Desktop/work/Hackverse/midprep-cloudphy/yolov7


In [19]:
from models.experimental import attempt_load
from utils.general import check_img_size, non_max_suppression, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.torch_utils import time_synchronized
from utils.datasets import letterbox

In [26]:
def detect(model, source):

    device = 'cpu'
    imgsz = 640
    img = source


    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check img_size TODO

    names = model.module.names if hasattr(model, 'module') else model.names
    old_img_w = old_img_h = imgsz
    old_img_b = 1
    im0 = img

        # Convert
    img = letterbox(img, imgsz, stride=stride)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)


    img = torch.from_numpy(img).to(device)
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
        pred = model(img)[0]
    # Apply NMS
    pred = non_max_suppression(pred, 0.25, 0.45, classes=[0,1,2,3,4,5])

    # Process detections
    outputs = []
    for i, det in enumerate(pred):  # detections per image

        if len(det):

            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            for *xyxy, conf, cls in reversed(det):
                label = f'{names[int(cls)]}'
                out = [t.item() for t in xyxy]
                outputs.append((out, label, conf.item()))

    return outputs

def crop_image(out, img):
  x1 = int(out[0][0])
  x2 = int(out[0][2])
  y1 = int(out[0][1])
  y2 = int(out[0][3])
  print(x1, x2, y1, y2)
  # Cropping an image
  cropped_image = img[y1:y2, x1:x2]

  return cropped_image

In [6]:
!pip install --upgrade --no-cache-dir gdown --quiet
!gdown 1qXBtVeOpZjeY-AduqtyTtu22y6RytGeO
!gdown 1Ni_w3QCucfHI271ASUjk7k04GZseGJcz

Downloading...
From (uriginal): https://drive.google.com/uc?id=1qXBtVeOpZjeY-AduqtyTtu22y6RytGeO
From (redirected): https://drive.google.com/uc?id=1qXBtVeOpZjeY-AduqtyTtu22y6RytGeO&confirm=t&uuid=5557e4e1-01d7-4008-9bf5-1591fec3df55
To: /Users/jayanth.kumar/Desktop/work/Hackverse/GTSRB/yolov7/classify.pt
100%|██████████████████████████████████████| 74.8M/74.8M [00:06<00:00, 11.0MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1Ni_w3QCucfHI271ASUjk7k04GZseGJcz
From (redirected): https://drive.google.com/uc?id=1Ni_w3QCucfHI271ASUjk7k04GZseGJcz&confirm=t&uuid=b3263093-b03e-422e-9671-4c5822d041eb
To: /Users/jayanth.kumar/Desktop/work/Hackverse/GTSRB/yolov7/segmentation.pt
100%|██████████████████████████████████████| 74.8M/74.8M [00:09<00:00, 8.07MB/s]


In [23]:
model1 = attempt_load('/Users/jayanth.kumar/Desktop/work/Hackverse/midprep-cloudphy1/new1/midprep-cloudphy/segmentation.pt')
model2 = attempt_load('/Users/jayanth.kumar/Desktop/work/Hackverse/midprep-cloudphy1/new1/midprep-cloudphy/classify.pt')
reader = easyocr.Reader(['en'])

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [21]:
!python detect.py --weights /Users/jayanth.kumar/Desktop/work/Hackverse/midprep-cloudphy1/new1/midprep-cloudphy/classify.pt --conf 0.25 --img-size 640 --source /Users/jayanth.kumar/Desktop/work/Hackverse/midprep-cloudphy/seg-split/train/images/hcgbhavnagar_micu_mon--403_2022_5_18_9_20_0.jpeg

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='/Users/jayanth.kumar/Desktop/work/Hackverse/midprep-cloudphy/seg-split/train/images/hcgbhavnagar_micu_mon--403_2022_5_18_9_20_0.jpeg', update=False, view_img=False, weights=['/Users/jayanth.kumar/Desktop/work/Hackverse/midprep-cloudphy1/new1/midprep-cloudphy/classify.pt'])
YOLOR 🚀 v0.1-122-g3b41c2c torch 2.0.0 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
/Users/jayanth.kumar/opt/anaconda3/envs/cloudphy/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors,

In [24]:
import cv2
def inference(image_path:str):
  '''
  Function responsible for inference.
  Args: 
    image_path: str, path to image file. eg. "input/aveksha_micu_mon--209_2023_1_17_12_0_34.jpeg"
  Returns:
    result: dict, final output dictionary. eg. {"HR":"80", "SPO2":"98", "RR":"15", "SBP":"126", "DBP":"86"}
  '''
  result = {}
  img = cv2.imread(image_path)
  
  out1 = detect(model1, img)
  out1_img = crop_image(out1[0], img)
  cv2.imwrite("./outpic.jpeg", out1_img)

  out2 = detect(model2, out1_img)
  temp = {}

  
  
  for out in out2:
    out2_img = crop_image(out, out1_img)

    r = reader.readtext(out2_img)
    if(not r):
      continue
    r = r[0]

    if(r[2] < 0.2):
      continue

    tt = temp.get(out[1])
    if(tt):
      if tt['class_conf'] > out[2]:
        continue


    temp[out[1]] = {
          'class_conf': out[2],
          'value': re.sub("[^0-9]", "", r[1]),
          'value_conf': r[2]
      }

  l = ['HR', 'SPO2', 'RR', 'SBP', 'DBP', 'MAP']

  for i in l:
    if(not temp.get(i)):
      continue

    result[i] = temp[i]['value']

  return result

In [27]:
inference("/Users/jayanth.kumar/Desktop/work/Hackverse/midprep-cloudphy/seg-split/val/images/hcgbhavnagar_micu_mon--403_2022_5_18_13_20_1.jpeg")

345 836 157 498
67 100 329 341
145 171 323 341
387 419 47 77
415 450 252 282


{'HR': '67', 'RR': '10'}

In [17]:
from PIL import Image
import numpy as np

def temp1(image_path:str):
  '''
  Function responsible for inference.
  Args: 
    image_path: str, path to image file. eg. "input/aveksha_micu_mon--209_2023_1_17_12_0_34.jpeg"
  Returns:
    result: dict, final output dictionary. eg. {"HR":"80", "SPO2":"98", "RR":"15", "SBP":"126", "DBP":"86"}
  '''
  result = {}
  img = cv2.imread(image_path)
  
  out1 = detect(model1, img)
  out1_img = crop_image(out1[0], img)

  img = Image.fromarray(out1_img, 'RGB')
  img.save('my.png')
  img.show()
  
  out2 = detect(model2, out1_img)
  temp = {}
  
  for out in out2:
    out2_img = crop_image(out, out1_img)
    print(type(out2_img))
    img = Image.fromarray(out2_img, 'RGB')
    img.save('my.png')
    img.show()

temp1("/Users/jayanth.kumar/Desktop/work/Hackverse/midprep-cloudphy/seg-split/val/images/hcgbhavnagar_micu_mon--403_2022_5_18_13_20_1.jpeg")

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
